In [1]:
!python --version

Python 3.6.9


In [2]:
# !pip install -q -U pip
# !pip install -q numpy
# !pip install -q pandas
!pip install -q ckiptagger
# !pip install -q tqdm
# !pip install -q tensorflow==1.14.0
# !pip install -q ipywidgets
# !pip install -q matplotlib

In [3]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

In [4]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [5]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [6]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!cp ./drive/MyDrive/NLP_data/data.7z .

In [9]:
!7z x data.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 3659454979 bytes (3490 MiB)

Extracting archive: data.7z
--
Path = data.7z
Type = 7z
Physical Size = 3659454979
Headers Size = 691
Method = LZMA2:26
Solid = +
Blocks = 2

  0%      0% 8 - D07data/embedding.7z                               1% 8 - D07data/embedding.7z                               2% 8 - D07data/embedding.7z                               3% 8 - D07data/embedding.7z                               4% 8 - D07data/embedding.7z                

In [10]:
ws = WS('./D07data/')
pos = POS('./D07data/')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [11]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [12]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

# 尋找降維的詞向量：PPMI + SVD

In [13]:
word2index = {}
index2word = {}
n = 0
for index in train_title_cuts:
    for word, flag in train_title_cuts[index]:
        if word in word2index:
            continue
        word2index[word] = n 
        index2word[n] = word
        n += 1

In [14]:
len(word2index)

6690

如果使用one-hot就需要這麼大的維度的詞向量

In [16]:
# 建立Co-Matrix

vocab_size = len(word2index)
co_matrix = np.zeros(shape=(vocab_size, vocab_size), dtype=np.int32)

window_size = 1
# YOUR CODE HERE
for pairs in train_title_cuts.values():
  words, _ = list(zip(*pairs))
  for idx, word in enumerate(words):
    left_idx = idx - window_size if idx - window_size >= 0 else 0
    left_words = words[left_idx: idx]
    word_id = word2index[word]
    for left_word in left_words:
      left_id = word2index[left_word]
      co_matrix[word_id, left_id] += 1
      co_matrix[left_id, word_id] += 1

# END YOUR CODE

In [21]:
co_matrix

array([[0, 2, 0, ..., 0, 0, 0],
       [2, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [23]:
# 建立PPMI

def get_ppmi(co_matrix: np.ndarray, eps: float=1e-8):
    # YOUR CODE HERE
    M = np.zeros_like(co_matrix, dtype = np.float32)
    N = np.sum(co_matrix)
    S = np.sum(co_matrix,axis = 0)
    total = co_matrix.shape[0] * co_matrix.shape[1]

    cnt = 0
    for i in tqdm(range(co_matrix.shape[0])):
      for j in range(co_matrix.shape[1]):
        pmi = np.log2(co_matrix[i, j] * N/ (S[i] * S[j] + eps))
        M[i, j] = max(0, pmi)

    # END YOUR CODE
    return M

ppmi = get_ppmi(co_matrix)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log2
  del sys.path[0]


In [ ]:
ppmi

array([[ 0.      ,  9.267157,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 9.267157,  0.      , 11.515084, ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      , 11.515084,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       ...,
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]], dtype=float32)

In [24]:
# 進行SVD分解，並得到降維的詞向量

from sklearn.decomposition import TruncatedSVD

# 使用`TruncatedSVD`進行降維，降維到dim=1000
# YOUR CODE HERE
svd = TruncatedSVD(n_components = 1000, n_iter = 10)
word_vectors = svd.fit_transform(ppmi)

# END YOUR CODE

In [25]:
word_vectors.shape

(6690, 1000)

# 新的詞向量 + Group mean vector: 測試

In [26]:
excluded_flags = [
    'Nh', 'Nep', 'Nes', 'DE', 'T', 'P', 'V_2', 'SHI',
    'Dfa', 'Dfb', 'Da', 'Di', 'Dk',
    'Caa', 'Cab', 'Cba', 'Cbb',
    'COLONCATEGORY', 'COMMACATEGORY', 'DASHCATEGORY', 'DOTCATEGORY', 'ETCCATEGORY', 'EXCLAMATIONCATEGORY',
    'PARENTHESISCATEGORY', 'PAUSECATEGORY', 'PERIODCATEGORY', 'QUESTIONCATEGORY', 'SEMICOLONCATEGORY',
    'SPCHANGECATEGORY', 'WHITESPACE'
]

In [27]:
train_svd_vectors = {}
for index, pairs in train_title_cuts.items():
    selected_word_vectors = []
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            selected_word_vectors.append(word_vectors[word2index[word], :])
    vector = np.sum(selected_word_vectors, axis=0)
    if np.sum(np.square(vector)) == 0:
        continue
    train_svd_vectors[index] = vector
    

In [28]:
test_svd_vectors = {}
for index, pairs in test_title_cuts.items():
    selected_word_vectors = []
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            selected_word_vectors.append(word_vectors[word2index[word], :])
    vector = np.sum(selected_word_vectors, axis=0)
    if np.sum(np.square(vector)) == 0:
        continue
    test_svd_vectors[index] = vector

In [29]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, vector in sorted(train_svd_vectors.items()):
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)

In [30]:
def cosine_similarity(bow1, bow2):
    len_bow1 = np.sqrt(np.sum(np.square(bow1)))
    len_bow2 = np.sqrt(np.sum(np.square(bow2)))
    return np.sum(bow1 * bow2) / (len_bow1 * len_bow2)

In [31]:
classification = {news_class: [] for news_class in all_news_class}
for index, vector in sorted(test_svd_vectors.items()):
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [32]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 62, '遊戲': 10, '旅遊': 9, '財經': 8, '科技': 5, '農業': 4})
predict 財經 :  Counter({'財經': 62, '科技': 24, '農業': 15, '體育': 8, '遊戲': 8, '旅遊': 7})
predict 科技 :  Counter({'科技': 52, '體育': 15, '財經': 14, '農業': 9, '遊戲': 9, '旅遊': 8})
predict 旅遊 :  Counter({'旅遊': 58, '農業': 12, '科技': 5, '財經': 4, '體育': 2, '遊戲': 2})
predict 農業 :  Counter({'農業': 58, '旅遊': 7, '體育': 4, '財經': 4, '遊戲': 4, '科技': 2})
predict 遊戲 :  Counter({'遊戲': 67, '科技': 11, '旅遊': 9, '財經': 8, '體育': 7, '農業': 1})
